# Coroutines, part 2

We saw this code last time, this was the version in which we handle `ValueError`: the exception handling code runs and advances us to the next yield.

In [1]:
def averager2(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        try:
            term = yield average
        except ValueError:
            term=0
        total += term
        count += 1
        average = total/count

In [2]:
av=averager2()
next(av)
av.throw(ValueError("yes"))

0.0

In [3]:
av.send(5)

2.5

In [4]:
av.close()

#### Returning a result

Notice here that we dont yield a running average, instead doing it as the return. So now we must create a termination condition. as you can see, if we send in a None, we achieve this by breaking the loop.

In [5]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [6]:
av=averager()

In [7]:
next(av)
av.send(5)

In [8]:
av.send(3)

In [9]:
next(av)#seems to be same as av.send(None)

StopIteration: Result(count=2, average=4.0)

It might seem strange that the return value of the coroutine is sent back on the exception. The reason for this is that we want to preserve the semantics of generator objects which come to us from a totally different use case of iteration: the raising of `StopIteration` when exhausted.

This will be important a bit later when we see `yield from`. `StopIteration` is handled transparently in loops; similarly, in `yield from` this also happens: indeed we consume the `StopIteration`, and the value sent on it: the result, becomes the ultimate value of the `yield from` expression itself.

### How Python generators work


#### Frames are allocated on the heap!

The standard Python interpreter is written in C. The C function that executes a Python function is called, mellifluously, PyEval_EvalFrameEx. It takes a Python stack frame object and evaluates Python bytecode in the context of the frame.

![](http://aosabook.org/en/500L/crawler-images/function-calls.png)

But the Python stack frames it manipulates are on the heap. Among other surprises, this means a Python stack frame can outlive its function call. To see this interactively, save the current frame from within bar

In [10]:
import inspect
def foo():
    bar()
    
def bar():
    global frame
    frame = inspect.currentframe()

In [11]:
foo()

In [12]:
frame.f_code.co_name

'bar'

For kicks, we can see the frame hierarchy:

In [13]:
caller_frame = frame.f_back
caller_frame.f_code.co_name

'foo'

Now, to understand how generators work, we bring back some code to inspect stuff from before...

In [14]:

#from https://bitbucket.org/yaniv_aknin/pynards/src/c4b61c7a1798766affb49bfba86e485012af6d16/common/blog.py?at=default&fileviewer=file-view-default
import dis
import types

def get_code_object(obj, compilation_mode="exec"):
    if isinstance(obj, types.CodeType):
        return obj
    elif isinstance(obj, types.FrameType):
        return obj.f_code
    elif isinstance(obj, types.FunctionType):
        return obj.__code__
    elif isinstance(obj, str):
        try:
            return compile(obj, "<string>", compilation_mode)
        except SyntaxError as error:
            raise ValueError("syntax error in passed string") from error
    else:
        raise TypeError("get_code_object() can not handle '%s' objects" %
                        (type(obj).__name__,))

def diss(obj, mode="exec", recurse=False):
    _visit(obj, dis.dis, mode, recurse)

def ssc(obj, mode="exec", recurse=False):
    _visit(obj, dis.show_code, mode, recurse)

def _visit(obj, visitor, mode="exec", recurse=False):
    obj = get_code_object(obj, mode)
    visitor(obj)
    if recurse:
        for constant in obj.co_consts:
            if type(constant) is type(obj):
                print()
                print('recursing into %r:' % (constant,))
                _visit(constant, visitor, mode, recurse)


Lets write a generator to inspect...

In [127]:
def gen():
    k=1
    result = yield 1
    result2 = yield 2
    return 'done'

In [128]:
ssc(gen)

Name:              gen
Filename:          <ipython-input-127-ab93e57304d7>
Argument count:    0
Kw-only arguments: 0
Number of locals:  3
Stack size:        1
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR, NOFREE
Constants:
   0: None
   1: 1
   2: 2
   3: 'done'
Variable names:
   0: k
   1: result
   2: result2


setting the `GENERATOR` flag means that python remembers to create a generator, not a function.

In [129]:
diss(gen)

  2           0 LOAD_CONST               1 (1)
              3 STORE_FAST               0 (k)

  3           6 LOAD_CONST               1 (1)
              9 YIELD_VALUE
             10 STORE_FAST               1 (result)

  4          13 LOAD_CONST               2 (2)
             16 YIELD_VALUE
             17 STORE_FAST               2 (result2)

  5          20 LOAD_CONST               3 ('done')
             23 RETURN_VALUE


In [130]:
g=gen()

In [131]:
dir(g)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_frame',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

There is no `g.__code__` like in functions, but there is s `g.gi_code`.

In [132]:
g.gi_code.co_name

'gen'

In [133]:
g.gi_frame

In [136]:
diss(g.gi_frame.f_code)

  2           0 LOAD_CONST               1 (1)
              3 STORE_FAST               0 (k)

  3           6 LOAD_CONST               1 (1)
              9 YIELD_VALUE
             10 STORE_FAST               1 (result)

  4          13 LOAD_CONST               2 (2)
             16 YIELD_VALUE
             17 STORE_FAST               2 (result2)

  5          20 LOAD_CONST               3 ('done')
             23 RETURN_VALUE


All generators point to the same code object. But each has its own `PyFrameObject`, a frame on the call stack, except that this frame is NOT on any call stack, but rather, in heap memory, waiting to be used.

![](http://aosabook.org/en/500L/crawler-images/generator.png)

Look at `f_lasti`, which is the last instruction in the bytecode. This is not particular to generators. Generator isnt started yet.

In [24]:
g.gi_frame.f_lasti, g.gi_running

(-1, False)

In [25]:
next(g)

1

In [26]:
g.gi_frame.f_lasti, g.gi_running

(3, False)

Because the generators stack frame was actually never put on the call stack, it can be resumed at any time, and by any caller. Its not stuck to the FILO nature of regular function execution.



In [27]:
g.send('a')

result of yield a


2

In [28]:
g.gi_frame.f_locals

{'result': 'a'}

In [29]:
g.send('b')

result of 2nd yield b


StopIteration: done

So, lets recap

- a generator can pause at a yield
- can be resumed with a new value thrown in
- can return a value.


### `yield from`

We saw it first some time back when we used it in a generator:

In [137]:
def gen2():
    yield from 'AB'
    yield from range(3)

In [138]:
list(gen2())

['A', 'B', 0, 1, 2]

The critical point about `yield from` is this: **when a generator `delegen` (delegating generator) calls `yield from subgen()`, `subgen` takes over and will yield values to the caller of `delegen`. Meanwhile `delgen` blocks until `subgen` terminates**.

Here is the subgenerator: our usual averaging function. Its a standard generator, setting up some kind of iterative process...

In [139]:
from collections import namedtuple
Result = namedtuple('Result', 'count average')
def averager_subgen():
    print("    |sg>starting subgen")
    total = 0.0
    count = 0 
    average = None 
    while True:
        print("    |sg>  average yielded out", average)
        term = yield average
        print("    |sg>  term sent in", term)
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

A delegating generator is one that does the `yield from`. It is a strange beast. It opens up a **bi-directional channel** from the outermost caller to the innermost sub-generator. Values can be thrown in and exceptions thrown out right through this channel. Here we create a channel with only one intermediary. But there is no reason there cant be many: we could have set up our online average and median calculations this way.

In [140]:
def average_my_values_delegen():
    while True:
        print('  |dg>+++++++++++++++')
        agen = averager_subgen()
        print("  |dg>created a new averager",id(agen))
        overall_av = yield from agen
        #yield from consumes all the values, like a list
        #not an individual value
        print("  |dg>now", overall_av)
    print("OVERALL AV", overall_av)
    return overall_av


In [147]:
def average_my_values_delegen_simple():
    print('  |dg>+++++++++++++++')
    agen = averager_subgen()
    print("  |dg>created a new averager",id(agen))
    overall_av = yield from agen
    #yield from consumes all the values, like a list
    #not an individual value
    print(" done with yield-from |dg>now", overall_av)
    print("OVERALL AV", overall_av)
    return overall_av

In [148]:
values_to_send=[1,2,3,4,5,6]

In [149]:
print("1. creating delegating generator")
delegating_gen = average_my_values_delegen_simple()
print("2. priming till yield from by sending in None")
next(delegating_gen)#priming
print("3. in loop after first yield, None sent in")
for value in values_to_send:
    print(">>sending term",value)
    out = delegating_gen.send(value)
    print('<<getting running average', out)
print("4. Sending in None to terminate")
out = delegating_gen.send(None)
print('<< OUTSIDE LOOP getting running average', out)
print("5. DONE")

1. creating delegating generator
2. priming till yield from by sending in None
  |dg>+++++++++++++++
  |dg>created a new averager 4389166864
    |sg>starting subgen
    |sg>  average yielded out None
3. in loop after first yield, None sent in
>>sending term 1
    |sg>  term sent in 1
    |sg>  average yielded out 1.0
<<getting running average 1.0
>>sending term 2
    |sg>  term sent in 2
    |sg>  average yielded out 1.5
<<getting running average 1.5
>>sending term 3
    |sg>  term sent in 3
    |sg>  average yielded out 2.0
<<getting running average 2.0
>>sending term 4
    |sg>  term sent in 4
    |sg>  average yielded out 2.5
<<getting running average 2.5
>>sending term 5
    |sg>  term sent in 5
    |sg>  average yielded out 3.0
<<getting running average 3.0
>>sending term 6
    |sg>  term sent in 6
    |sg>  average yielded out 3.5
<<getting running average 3.5
4. Sending in None to terminate
    |sg>  term sent in None
 done with yield-from |dg>now Result(count=6, average=3.5)
OV

StopIteration: Result(count=6, average=3.5)

In [151]:
print("1. creating delegating generator")
delegating_gen = average_my_values_delegen()
print("2. priming till yield from by sending in None")
next(delegating_gen)#priming
print("3. in loop after first yield, None sent in")
for value in values_to_send:
    print(">>sending term",value)
    out = delegating_gen.send(value)
    print('<<getting running average', out)
print("4. Sending in None to terminate")
out = delegating_gen.send(None)
print('<<OUTSIDE LOOP getting running average', out)
print("5. DONE")

1. creating delegating generator
2. priming till yield from by sending in None
  |dg>+++++++++++++++
  |dg>created a new averager 4389076136
    |sg>starting subgen
    |sg>  average yielded out None
3. in loop after first yield, None sent in
>>sending term 1
    |sg>  term sent in 1
    |sg>  average yielded out 1.0
<<getting running average 1.0
>>sending term 2
    |sg>  term sent in 2
    |sg>  average yielded out 1.5
<<getting running average 1.5
>>sending term 3
    |sg>  term sent in 3
    |sg>  average yielded out 2.0
<<getting running average 2.0
>>sending term 4
    |sg>  term sent in 4
    |sg>  average yielded out 2.5
<<getting running average 2.5
>>sending term 5
    |sg>  term sent in 5
    |sg>  average yielded out 3.0
<<getting running average 3.0
>>sending term 6
    |sg>  term sent in 6
    |sg>  average yielded out 3.5
<<getting running average 3.5
4. Sending in None to terminate
    |sg>  term sent in None
  |dg>now Result(count=6, average=3.5)
  |dg>+++++++++++++++


Notice that the two delegating generators behave a bit differently. In `delegen_simple` a `StopIteration` is propagated up. In `delegen` this exception is handled. How did this happen? 

The reason is that in the `simple` case, the delegating generator is still a generator and it hits the `return` statement, which then causes a `StopIteration` with the return value. So the exception comes from the delegating generator. (The `overall_av = yield from agen` ends cleanly: delegating generators, just like list iteration, terminate their subgenerators properly)

In the non-simple case, the delegating generator instance never returns and thus no StopIteration. What happens instead is that a new sub-generator is created and suspended after yielding out None. The delegating generator is also suspended until we send in something and we get our repl. The return never reached.

In [152]:
delegating_gen.send(None)

    |sg>  term sent in None
  |dg>now Result(count=0, average=None)
  |dg>+++++++++++++++
  |dg>created a new averager 4364040240
    |sg>starting subgen
    |sg>  average yielded out None


And ad-infinitum unless we explicitly close(send in GeneratorExit and handle it thrown back out)

In [153]:
delegating_gen.close()

Lets make a summary of the rules of delegation using `yield from`.

#### Summary of the rules of `yield from`

- yielded values from subgen go to caller (as if the delegating generator was yielding upward)
- sent in values from caller go to subgen, is not None, they trigger `send` on the subgen
- delegating gen blocks until subgen stop-iterates and collects its return value: the termination is clean, like in a list iteration.
- -f `next` is called, or `None` is senr in, the subgens `__next__()` is called, which can result in a `StopIteration`, which resumes the delegating gen at the `yield from` statement.
- return BLA in gen => StopIteration(BLA) is raised on exit from that gen. This is true of the subgen, and this argument becomes the value of the `yield from` expression
- notice we didnt prime the subgen. `yield from` does this for us, advancing us to the first `yield` in the subgen.

There are some subtleties associated with exception handling. Quoting from Fluent:

> 
- Exceptions other than GeneratorExit thrown into the delegating generator are passed to the throw() method of the subgenerator. If the call raises StopItera tion, the delegating generator is resumed. Any other exception is propagated to the delegating generator.
- If a GeneratorExit exception is thrown into the delegating generator, or the close() method of the delegating generator is called, then the close() method of the subgenerator is called if it has one. If this call results in an exception, it is propagated to the delegating generator. Otherwise, GeneratorExit is raised in the delegating generator.

#### Implementing `yield from`

Perhaps seeing a simplified implementation of `yield from` might help. From fluent:

```python
_i = iter(EXPR) #subgenerator
try:
    _y = next(_i) #do priming automatically
except StopIteration as _e:
    _r = _e.value #get value of StopIteration as result on exception
else: # if first next was successful, then
    while 1: #sit in a loop, blocking us
        _s = yield _y #yield the current value from the subgen upwards; then wait for a value _s from caller
        try:
            _y = _i.send(_s)#send subgen _s
            #store what it yields in _y, go back to top ofloop
        except StopIteration as _e: #if subgen StopIterated
            _r = _e.value#set result and exit loop, unblocking us
            break
RESULT = _r
```


### How does `yield from` work?

In [46]:
def subgen():
    result1 = yield 1
    print('sg yield 1: ', result1)
    result2 = yield 2
    print('sg yield 2: ', result2)
    return 'done'
sg=subgen()

To call this generator from another generator, delegate to it with yield from:


In [47]:
def delegen():
    sg = subgen()
    rv = yield from sg
    print('return value of yield-from',rv)

In [48]:
dg = delegen()

In [49]:
dis.dis(dg)

  2           0 LOAD_GLOBAL              0 (subgen)
              3 CALL_FUNCTION            0 (0 positional, 0 keyword pair)
              6 STORE_FAST               0 (sg)

  3           9 LOAD_FAST                0 (sg)
             12 GET_YIELD_FROM_ITER
             13 LOAD_CONST               0 (None)
             16 YIELD_FROM
             17 STORE_FAST               1 (rv)

  4          20 LOAD_GLOBAL              1 (print)
             23 LOAD_CONST               1 ('return value of yield-from')
             26 LOAD_FAST                1 (rv)
             29 CALL_FUNCTION            2 (2 positional, 0 keyword pair)
             32 POP_TOP
             33 LOAD_CONST               0 (None)
             36 RETURN_VALUE


In [50]:
dis.show_code(dg)

Name:              delegen
Filename:          <ipython-input-47-32124c079646>
Argument count:    0
Kw-only arguments: 0
Number of locals:  2
Stack size:        3
Flags:             OPTIMIZED, NEWLOCALS, GENERATOR, NOFREE
Constants:
   0: None
   1: 'return value of yield-from'
Names:
   0: subgen
   1: print
Variable names:
   0: sg
   1: rv


In [51]:
dg.send(None)

1

In [52]:
dg.gi_frame.f_lasti

15

In [53]:
dg.send(5)

sg yield 1:  5


2

In [54]:
dg.gi_frame.f_lasti

15

We are stuck in the bytecode at the yield-from...

In [55]:
dg.send("hello")

sg yield 2:  hello
return value of yield-from done


StopIteration: 

Notice that the delegating generator does not advance. Meanwhile the inner generator advances from one yield statement to the next. From the point of view of the REPL (us as the client in the REPL), we cant tell where the values yielded us are coming from, and where the values we send are going. Ditto inside the subgen: we dont know where the values are coming from. 


### An example: grouping using `yield from`.

This example is taken directly from Fluent and should now be easy to understand...

In [155]:
d={'a':[1,2,4,5],'b':[11,12,13,14]}

In [156]:
#subgenerator
def averager(): 
    total = 0.0
    count = 0 
    average = None 
    while True:
        term = yield
        print("got in av", term)
        if term is None:
            break
        total += term
        count += 1
        average = total/count
    return Result(count, average)

In [157]:
#delegating generator
def grouper(results, key): 
    while True:
        avg = averager()
        print("new averager", key, avg)
        results[key] = yield from avg#bi-directional channel
        print("got after yield from", results[key])

In [158]:
def main(data): 
    results = {}
    for key, values in data.items():
        print("starting", key)
        group = grouper(results, key) #averager "instance" created here, 
        next(group)#and advanced
        #sub-generator is automatically primed
        for value in values:
            print("sent", value)
            group.send(value) 
        print("done sending for ",key)
        group.send(None) # important!
        print("here")
    print(results)

In [159]:
main(d)

starting b
new averager b <generator object averager at 0x1059fa5c8>
sent 11
got in av 11
sent 12
got in av 12
sent 13
got in av 13
sent 14
got in av 14
done sending for  b
got in av None
got after yield from Result(count=4, average=12.5)
new averager b <generator object averager at 0x1059fad58>
here
starting a
new averager a <generator object averager at 0x1059fa8e0>
sent 1
got in av 1
sent 2
got in av 2
sent 4
got in av 4
sent 5
got in av 5
done sending for  a
got in av None
got after yield from Result(count=4, average=3.0)
new averager a <generator object averager at 0x1059fad58>
here
{'b': Result(count=4, average=12.5), 'a': Result(count=4, average=3.0)}
